In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, datetime
import holidays
v_directory = 'Artigo/Analise de Sentimento/TimeSeriesArticle'

# Analysis for S&P500 index

### Are there missing values on the original data?

In [2]:
v_sp500 = pd.read_csv('S&P500.csv', parse_dates=True)
v_sp500.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
5079,2020-03-12,2630.860107,2660.949951,2478.860107,2480.639893,2480.639893,8829380000
5080,2020-03-13,2569.989990,2711.330078,2492.370117,2711.020020,2711.020020,8258670000
5081,2020-03-16,2508.590088,2562.979980,2380.939941,2386.129883,2386.129883,7781540000
5082,2020-03-17,2425.659912,2553.929932,2367.040039,2529.189941,2529.189941,8358500000
5083,2020-03-18,2436.500000,2453.570068,2280.520020,2398.100098,2398.100098,8755780000


In [3]:
len(v_sp500)

5084

In [4]:
at_least_one_value_missing = v_sp500.apply(lambda x: (len(x)-x.count())>=1, axis=1)

In [5]:
at_least_one_value_missing.value_counts()

False    5084
dtype: int64

#### There are no missing values on the original data.
### Are there missing days on the original data?

In [88]:
business_dates = pd.bdate_range(start='2000-01-03', end='2020-03-18').date
business_dates = business_dates.tolist()
#dates

In [89]:
num_business_days = len(business_dates)
num_business_days

5273

In [90]:
difference = num_business_days-len(v_sp500)
difference

189

#### There are 189 missing business days on the data.
### Are those holidays? 
#### It is possible, since there are 20 years of data and approximately 10 holidays per year in the USA, and a few might have happened in weekends (that are already not being considered)

In [91]:
dates = list()
years = list(range(2000,2021))
us_holidays = holidays.UnitedStates(years = years, expand = False, observed=False)
#us_holidays

In [92]:
for date in business_dates:
    if(date not in list(us_holidays.keys())):
        dates.append(date)
        #print("Not Holiday", date) 
    else:
        print("Holiday", date) 

Holiday 2000-01-17
Holiday 2000-02-21
Holiday 2000-05-29
Holiday 2000-07-04
Holiday 2000-09-04
Holiday 2000-10-09
Holiday 2000-11-23
Holiday 2000-12-25
Holiday 2001-01-01
Holiday 2001-01-15
Holiday 2001-02-19
Holiday 2001-05-28
Holiday 2001-07-04
Holiday 2001-09-03
Holiday 2001-10-08
Holiday 2001-11-22
Holiday 2001-12-25
Holiday 2002-01-01
Holiday 2002-01-21
Holiday 2002-02-18
Holiday 2002-05-27
Holiday 2002-07-04
Holiday 2002-09-02
Holiday 2002-10-14
Holiday 2002-11-11
Holiday 2002-11-28
Holiday 2002-12-25
Holiday 2003-01-01
Holiday 2003-01-20
Holiday 2003-02-17
Holiday 2003-05-26
Holiday 2003-07-04
Holiday 2003-09-01
Holiday 2003-10-13
Holiday 2003-11-11
Holiday 2003-11-27
Holiday 2003-12-25
Holiday 2004-01-01
Holiday 2004-01-19
Holiday 2004-02-16
Holiday 2004-05-31
Holiday 2004-09-06
Holiday 2004-10-11
Holiday 2004-11-11
Holiday 2004-11-25
Holiday 2005-01-17
Holiday 2005-02-21
Holiday 2005-05-30
Holiday 2005-07-04
Holiday 2005-09-05
Holiday 2005-10-10
Holiday 2005-11-11
Holiday 2005

In [93]:
len(dates)

5092

In [94]:
num_business_days-len(dates)

181

In [95]:
len(v_sp500)-len(dates)

-8

### Dates present in sp500 but not on our list date

In [97]:
count = 0
for date in v_sp500['Date']:
    if datetime.strptime(date, '%Y-%m-%d').date() not in dates:
        print(datetime.strptime(date, '%Y-%m-%d').date())
        count +=1
         
print(count, "dates are missing from our dates list")

2000-10-09
2001-10-08
2002-10-14
2002-11-11
2003-10-13
2003-11-11
2004-10-11
2004-11-11
2005-10-10
2005-11-11
2006-10-09
2007-10-08
2008-10-13
2008-11-11
2009-10-12
2009-11-11
2010-10-11
2010-11-11
2011-10-10
2011-11-11
2012-10-08
2013-10-14
2013-11-11
2014-10-13
2014-11-11
2015-10-12
2015-11-11
2016-10-10
2016-11-11
2017-10-09
2018-10-08
2019-10-14
2019-11-11
33 dates are missing from our dates list


#### they are all holidays that still had the index informed. No problem here.


### Dates present on our date list but missing from sp500

In [98]:
count = 0
for date in dates:
    if date.strftime('%Y-%m-%d') not in v_sp500['Date'].unique():
        print(date.strftime('%Y-%m-%d'))
        count += 1
print(count, " dates missing from our date list")

2000-04-21
2001-04-13
2001-09-11
2001-09-12
2001-09-13
2001-09-14
2002-03-29
2003-04-18
2004-04-09
2004-06-11
2004-07-05
2004-12-24
2005-03-25
2005-12-26
2006-01-02
2006-04-14
2007-01-02
2007-04-06
2008-03-21
2009-04-10
2009-07-03
2010-04-02
2010-07-05
2010-12-24
2011-04-22
2011-12-26
2012-01-02
2012-04-06
2012-10-29
2012-10-30
2013-03-29
2014-04-18
2015-04-03
2015-07-03
2016-03-25
2016-12-26
2017-01-02
2017-04-14
2018-03-30
2018-12-05
2019-04-19
41  dates missing from our date list


#### Those dates are actualy missing values from the sp500 index. They are not holidays or weekends.
#### Those dates should be imputed.

# Analysis for vix index

### Are there missing values on the original data?

In [105]:
v_vix = pd.read_csv('VIX.csv', parse_dates=True)
v_vix.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
5079,2020-03-12,61.459999,76.830002,59.910000,75.470001,75.470001,0
5080,2020-03-13,65.070000,77.570000,55.169998,57.830002,57.830002,0
5081,2020-03-16,57.830002,83.559998,57.830002,82.690002,82.690002,0
5082,2020-03-17,82.690002,84.830002,70.370003,75.910004,75.910004,0
5083,2020-03-18,69.370003,85.470001,69.370003,76.449997,76.449997,0


In [106]:
len(v_vix)

5084

In [107]:
at_least_one_value_missing = v_vix.apply(lambda x: (len(x)-x.count())>=1, axis=1)

In [108]:
at_least_one_value_missing.value_counts()

False    5084
dtype: int64

#### There are no missing values on the original data.
### Are there missing days on the original data?

In [109]:
business_dates = pd.bdate_range(start='2000-01-03', end='2020-03-18').date
business_dates = business_dates.tolist()

In [110]:
num_business_days = len(business_dates)
num_business_days

5273

In [111]:
difference = num_business_days-len(v_sp500)
difference

189

#### There are 189 missing business days on the data.
### Are those holidays? 
#### It is possible, since there are 20 years of data and approximately 10 holidays per year in the USA, and a few might have happened in weekends (that are already not being considered)

In [112]:
dates = list()
years = list(range(2000,2021))
us_holidays = holidays.UnitedStates(years = years, expand = False, observed=False)
#us_holidays

In [113]:
for date in business_dates:
    if(date not in list(us_holidays.keys())):
        dates.append(date)
        #print("Not Holiday", date) 
    else:
        print("Holiday", date) 

Holiday 2000-01-17
Holiday 2000-02-21
Holiday 2000-05-29
Holiday 2000-07-04
Holiday 2000-09-04
Holiday 2000-10-09
Holiday 2000-11-23
Holiday 2000-12-25
Holiday 2001-01-01
Holiday 2001-01-15
Holiday 2001-02-19
Holiday 2001-05-28
Holiday 2001-07-04
Holiday 2001-09-03
Holiday 2001-10-08
Holiday 2001-11-22
Holiday 2001-12-25
Holiday 2002-01-01
Holiday 2002-01-21
Holiday 2002-02-18
Holiday 2002-05-27
Holiday 2002-07-04
Holiday 2002-09-02
Holiday 2002-10-14
Holiday 2002-11-11
Holiday 2002-11-28
Holiday 2002-12-25
Holiday 2003-01-01
Holiday 2003-01-20
Holiday 2003-02-17
Holiday 2003-05-26
Holiday 2003-07-04
Holiday 2003-09-01
Holiday 2003-10-13
Holiday 2003-11-11
Holiday 2003-11-27
Holiday 2003-12-25
Holiday 2004-01-01
Holiday 2004-01-19
Holiday 2004-02-16
Holiday 2004-05-31
Holiday 2004-09-06
Holiday 2004-10-11
Holiday 2004-11-11
Holiday 2004-11-25
Holiday 2005-01-17
Holiday 2005-02-21
Holiday 2005-05-30
Holiday 2005-07-04
Holiday 2005-09-05
Holiday 2005-10-10
Holiday 2005-11-11
Holiday 2005

In [114]:
len(dates)

5092

In [115]:
num_business_days-len(dates)

181

In [116]:
len(v_sp500)-len(dates)

-8

### Dates present in sp500 but not on our list date

In [117]:
count = 0
for date in v_vix['Date']:
    if datetime.strptime(date, '%Y-%m-%d').date() not in dates:
        print(datetime.strptime(date, '%Y-%m-%d').date())
        count +=1
         
print(count, "dates are missing from our dates list")

2000-10-09
2001-10-08
2002-10-14
2002-11-11
2003-10-13
2003-11-11
2004-10-11
2004-11-11
2005-10-10
2005-11-11
2006-10-09
2007-10-08
2008-10-13
2008-11-11
2009-10-12
2009-11-11
2010-10-11
2010-11-11
2011-10-10
2011-11-11
2012-10-08
2013-10-14
2013-11-11
2014-10-13
2014-11-11
2015-10-12
2015-11-11
2016-10-10
2016-11-11
2017-10-09
2018-10-08
2019-10-14
2019-11-11
33 dates are missing from our dates list


#### they are all holidays that still had the index informed. No problem here.


### Dates present on our date list but missing from sp500

In [119]:
count = 0
for date in dates:
    if date.strftime('%Y-%m-%d') not in v_vix['Date'].unique():
        print(date.strftime('%Y-%m-%d'))
        count += 1
print(count, " dates missing from sp500 list")

2000-04-21
2001-04-13
2001-09-11
2001-09-12
2001-09-13
2001-09-14
2002-03-29
2003-04-18
2004-04-09
2004-06-11
2004-07-05
2004-12-24
2005-03-25
2005-12-26
2006-01-02
2006-04-14
2007-01-02
2007-04-06
2008-03-21
2009-04-10
2009-07-03
2010-04-02
2010-07-05
2010-12-24
2011-04-22
2011-12-26
2012-01-02
2012-04-06
2012-10-29
2012-10-30
2013-03-29
2014-04-18
2015-04-03
2015-07-03
2016-03-25
2016-12-26
2017-01-02
2017-04-14
2018-03-30
2018-12-05
2019-04-19
41  dates missing from sp500 list


#### Those dates are actualy missing values from the sp500 index. They are not holidays or weekends.
#### Those dates should be imputed.